## IMPORTS

In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
import os.path

from sklearn.model_selection import train_test_split

import tensorflow as tf

2025-05-07 18:01:51.959526: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746640912.247488      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746640912.324003      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
image_dir = Path('../input/a-large-scale-fish-dataset/Fish_Dataset/Fish_Dataset')

## Creating DF

In [3]:
# Get filepaths and labels
filepaths = list(image_dir.glob(r'**/*.png'))
labels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1], filepaths))

filepaths = pd.Series(filepaths, name='Filepath').astype(str)
labels = pd.Series(labels, name='Label')

# Concatenate filepaths and labels
image_df = pd.concat([filepaths, labels], axis=1)

# Drop GT images
image_df['Label'] = image_df['Label'].apply(lambda x: np.NaN if x[-2:] == 'GT' else x)
image_df = image_df.dropna(axis=0)

# Sample 200 images from each class
samples = []

for category in image_df['Label'].unique():
    category_slice = image_df.query("Label == @category")
    samples.append(category_slice.sample(200, random_state=1))

image_df = pd.concat(samples, axis=0).sample(frac=1.0, random_state=1).reset_index(drop=True)

In [4]:
image_df

,Filepath,Label
0,../input/a-large-scale-fish-dataset/Fish_Datas...,Gilt-Head Bream
1,../input/a-large-scale-fish-dataset/Fish_Datas...,Sea Bass
2,../input/a-large-scale-fish-dataset/Fish_Datas...,Red Mullet
3,../input/a-large-scale-fish-dataset/Fish_Datas...,Shrimp
4,../input/a-large-scale-fish-dataset/Fish_Datas...,Trout
...,...,...
1795,../input/a-large-scale-fish-dataset/Fish_Datas...,Trout
1796,../input/a-large-scale-fish-dataset/Fish_Datas...,Red Sea Bream
1797,../input/a-large-scale-fish-dataset/Fish_Datas...,Striped Red Mullet
1798,../input/a-large-scale-fish-dataset/Fish_Datas...,Black Sea Sprat


In [5]:
train_df, test_df = train_test_split(image_df, train_size=0.7, shuffle=True, random_state=1)

## Loading Images

In [6]:
train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input,
    validation_split=0.2
)

test_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input
)

In [7]:
train_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='training'
)

val_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='validation'
)

test_images = test_generator.flow_from_dataframe(
    dataframe=test_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=False
)

Found 1008 validated image filenames belonging to 9 classes.
Found 252 validated image filenames belonging to 9 classes.
Found 540 validated image filenames belonging to 9 classes.


## Load Pre Trained Model

In [8]:
pretrained_model = tf.keras.applications.MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet',
    pooling='avg'
)

pretrained_model.trainable = False

2025-05-07 18:03:30.577013: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


## Training

In [9]:
inputs = pretrained_model.input

x = tf.keras.layers.Dense(128, activation='relu')(pretrained_model.output)
x = tf.keras.layers.Dense(128, activation='relu')(x)

outputs = tf.keras.layers.Dense(9, activation='softmax')(x)


model = tf.keras.Model(inputs=inputs, outputs=outputs)


model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


history = model.fit(
    train_images,
    validation_data=val_images,
    epochs=100,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=3,
            restore_best_weights=True
        )
    ]
)

Epoch 1/100


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


32/32 ━━━━━━━━━━━━━━━━━━━━ 57s 1s/step - accuracy: 0.6010 - loss: 1.3003 - val_accuracy: 0.9762 - val_loss: 0.1391
Epoch 2/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 42s 1s/step - accuracy: 0.9863 - loss: 0.0712 - val_accuracy: 0.9921 - val_loss: 0.0628
Epoch 3/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 39s 1s/step - accuracy: 0.9970 - loss: 0.0259 - val_accuracy: 0.9643 - val_loss: 0.1030
Epoch 4/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 39s 1s/step - accuracy: 0.9961 - loss: 0.0135 - val_accuracy: 0.9921 - val_loss: 0.0484
Epoch 5/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 38s 1s/step - accuracy: 1.0000 - loss: 0.0053 - val_accuracy: 0.9802 - val_loss: 0.0474
Epoch 6/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 38s 1s/step - accuracy: 1.0000 - loss: 0.0039 - val_accuracy: 0.9802 - val_loss: 0.0514
Epoch 7/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 42s 1s/step - accuracy: 1.0000 - loss: 0.0023 - val_accuracy: 0.9802 - val_loss: 0.0509
Epoch 8/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 38s 1s/step - accuracy: 1.0000 - loss: 0.0020 - val_accuracy: 0.9802 - val_loss: 0.0

In [10]:
results = model.evaluate(test_images, verbose=0)

print("    Test Loss: {:.5f}".format(results[0]))
print("Test Accuracy: {:.2f}%".format(results[1] * 100))

    Test Loss: 0.02651
Test Accuracy: 99.44%


## All Models

In [20]:
import os
import numpy as np
import pandas as pd
from pathlib import Path

# Define your image directory
image_dir = Path('../input/a-large-scale-fish-dataset/Fish_Dataset/Fish_Dataset') # <--- change this to your dataset path

# Get filepaths and labels
filepaths = list(image_dir.glob(r'**/*.png'))
labels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1], filepaths))

filepaths = pd.Series(filepaths, name='Filepath').astype(str)
labels = pd.Series(labels, name='Label')

# Concatenate filepaths and labels
image_df = pd.concat([filepaths, labels], axis=1)

# Drop GT images
image_df['Label'] = image_df['Label'].apply(lambda x: np.NaN if x[-2:] == 'GT' else x)
image_df = image_df.dropna(axis=0)

# ✅ Sample up to 200 images per class — skip classes with fewer images
samples = []
min_required = 200  # number of samples per class

for category in image_df['Label'].unique():
    category_slice = image_df.query("Label == @category")
    if len(category_slice) >= min_required:
        samples.append(category_slice.sample(min_required, random_state=1))
    else:
        print(f"Skipping class '{category}' — only {len(category_slice)} images available.")

# ✅ Check if we have valid data
if not samples:
    raise ValueError("❌ No categories had enough images to include in the dataset.")

# ✅ Final dataset
image_df = pd.concat(samples, axis=0).sample(frac=1.0, random_state=1).reset_index(drop=True)


In [ ]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
import pandas as pd
import os
from tensorflow.keras.applications import VGG16, ResNet50, MobileNetV2, InceptionV3, EfficientNetB0

# Step 1: Split dataset into train and validation
train_df, val_df = train_test_split(image_df, test_size=0.2, random_state=42)

# Step 2: Set up ImageDataGenerator with preprocessing for each model
train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input,
)

val_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input,
)

# Load images and labels for training and validation
train_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=True,
    seed=42,
)

val_images = val_generator.flow_from_dataframe(
    dataframe=val_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=True,
    seed=42,
)

# Step 3: Define a function to create the models
def create_model(base_model):
    model = tf.keras.Model(inputs=base_model.input, outputs=base_model.output)
    x = tf.keras.layers.Dense(128, activation='relu')(model.output)
    x = tf.keras.layers.Dense(128, activation='relu')(x)
    outputs = tf.keras.layers.Dense(len(train_images.class_indices), activation='softmax')(x)
    final_model = tf.keras.Model(inputs=model.input, outputs=outputs)
    
    final_model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return final_model

# Step 4: Define models to test (VGG16, ResNet50, MobileNetV2, InceptionV3, EfficientNetB0)
models = {
    "VGG16": tf.keras.applications.VGG16(input_shape=(224, 224, 3), include_top=False, weights='imagenet', pooling='avg'),
    "ResNet50": tf.keras.applications.ResNet50(input_shape=(224, 224, 3), include_top=False, weights='imagenet', pooling='avg'),
    "MobileNetV2": tf.keras.applications.MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet', pooling='avg'),
    "InceptionV3": tf.keras.applications.InceptionV3(input_shape=(224, 224, 3), include_top=False, weights='imagenet', pooling='avg'),
    "EfficientNetB0": tf.keras.applications.EfficientNetB0(input_shape=(224, 224, 3), include_top=False, weights='imagenet', pooling='avg')
}

# Step 5: Train and evaluate each model
results = {}

for model_name, base_model in models.items():
    print(f"\nTraining {model_name}...")
    
    # Freeze base model layers
    base_model.trainable = False

    # Create model and train
    model = create_model(base_model)
    
    # Train the model
    history = model.fit(
        train_images,
        validation_data=val_images,
        epochs=100,
        callbacks=[
            tf.keras.callbacks.EarlyStopping(
                monitor='val_loss',
                patience=3,
                restore_best_weights=True
            )
        ]
    )
    
    # Evaluate model on validation data
    val_loss, val_accuracy = model.evaluate(val_images)
    results[model_name] = val_accuracy
    print(f"{model_name} - Validation Accuracy: {val_accuracy}")

# Step 6: Print the results
print("\nResults for all models:")
for model_name, accuracy in results.items():
    print(f"{model_name}: {accuracy:.4f}")


Found 1440 validated image filenames belonging to 9 classes.
Found 360 validated image filenames belonging to 9 classes.
58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step

Training VGG16...
Epoch 1/100


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


45/45 ━━━━━━━━━━━━━━━━━━━━ 536s 12s/step - accuracy: 0.2424 - loss: 2.1123 - val_accuracy: 0.5917 - val_loss: 1.6224
Epoch 2/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 531s 12s/step - accuracy: 0.6193 - loss: 1.4513 - val_accuracy: 0.7806 - val_loss: 0.9465
Epoch 3/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 567s 13s/step - accuracy: 0.8067 - loss: 0.8253 - val_accuracy: 0.7944 - val_loss: 0.6666
Epoch 4/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 531s 12s/step - accuracy: 0.8688 - loss: 0.5173 - val_accuracy: 0.9139 - val_loss: 0.4213
Epoch 5/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 531s 12s/step - accuracy: 0.9220 - loss: 0.3503 - val_accuracy: 0.9333 - val_loss: 0.3454
Epoch 6/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 533s 12s/step - accuracy: 0.9221 - loss: 0.2659 - val_accuracy: 0.9306 - val_loss: 0.2632
Epoch 7/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 568s 13s/step - accuracy: 0.9563 - loss: 0.1977 - val_accuracy: 0.9500 - val_loss: 0.1987
Epoch 8/100
45/45 ━━━━━━━━━━━━━━━━━━━━ 531s 12s/step - accuracy: 0.9791 - loss: 0.1555 - val_accuracy: 0.9556